In [1]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

# GPU 정보 출력
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())


GPU is available
NVIDIA GeForce RTX 3090
0


In [2]:
from tqdm.notebook import tqdm
tqdm().pandas()

0it [00:00, ?it/s]

In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [4]:
import numpy as np
np.bool = np.bool_

class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab 

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')



In [6]:
import pandas as pd


data = pd.read_csv("datasets/5차년도_2차.csv", encoding='cp949')
data.head()

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female


In [7]:
data['상황'].unique()

array(['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust',
       'fear'], dtype=object)

In [9]:
emotion_df = pd.read_csv('datasets/감성대화말뭉치(최종데이터)_Training.csv')
emotion_df['감정_대분류'].unique()

def concatenate_sentences(row):
    sentences = [row['사람문장1'], row['사람문장2']]
    return '[CLS]'.join(sentences)

# 각 감정별 리스트 생성 및 사람문장 이어붙이기
# 4가지 감정만 남기기
emotions = ['분노', '기쁨', '불안', '슬픔']
filtered_df = emotion_df[emotion_df['감정_대분류'].isin(emotions)]

filtered_df

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51621,51622,중년,여성,"대인관계(부부, 자녀)",해당없음,분노,툴툴대는,남편이 내 곁을 떠났어. 아무것도 못 해준 내가 실망스럽고 자꾸 눈물이 나.,속상하고 슬프시군요. 많이 힘드시겠어요.,내가 생전에 잘 못 해주고 아플 때 간호도 잘 못 해준 것 같아. 후회돼.,지금의 상황에서 어떻게 하는 것이 최선의 방법일까요?,나의 아픔을 나눌 수 있고 내가 모든 것을 말할 수 있는 사람을 찾아야겠어.,아픔을 나눌 수 있는 좋은 사람을 찾으셨으면 좋겠어요.
51622,51623,노년,남성,"건강,죽음",만성질환 무,분노,노여워하는,건강관리를 너무 안 해서 건강이 좋지 않아 졌어. 주변 사람들에게 폐 끼칠까 봐 걱정돼.,건강에 대한 문제 때문에 마음이 편치 않으시겠어요.,아내한테 병시중 시킬까 봐 미안하고 걱정돼. 건강을 챙기지 않은 내가 실망스러워.,지금의 상황에서 할 수 있는 최선의 행동은 무엇일까요?,이제부터라도 건강관리에 힘써서 더 악화되지 않게 해야겠어.,건강관리에 성공하시길 바랄게요!
51625,51626,노년,남성,재정,만성질환 무,분노,성가신,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.,지금 할 수 있는 가장 합리적인 행동은 무엇인가요?,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.
51626,51627,노년,여성,재정,만성질환 무,불안,초조한,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.,어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.


In [10]:
def concatenate_sentences(row):
    sentences = [row['사람문장1'], row['사람문장2'], row['사람문장3']]
    # NaN 값을 건너뛰고 문장을 이어 붙이기
    return '[CLS]'.join([str(sentence) for sentence in sentences if pd.notna(sentence)])

# 각 감정별 리스트 생성 및 사람문장 이어붙이기
concatenated_lists = []

for emotion in emotions:
    emotion_df = filtered_df[filtered_df['감정_대분류'] == emotion].copy()
    concatenated_sentences = emotion_df.apply(concatenate_sentences, axis=1)
    for sentence in concatenated_sentences:
        concatenated_lists.append([sentence, emotion])

# 결과 확인
for item in concatenated_lists[:10000]:  # 처음 10개 항목만 출력
    print(item)
    


['일은 왜 해도 해도 끝이 없을까? 화가 난다.[CLS]그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.', '분노']
['이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.[CLS]최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.', '분노']
['회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. [CLS]잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대하는 게 덜 스트레스받을 것 같아.', '분노']
['직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.[CLS]직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.', '분노']
['얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.[CLS]상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!', '분노']
['직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.[CLS]직장 상사한테 자주 지적을 받아. 그럴 때마다 이 업무는 나랑 맞지 않는 거 같이 느껴져.', '분노']
['성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.[CLS]부모님께 좀 더 시간을 달라고 해야겠어. 진로 테스트를 받아볼까 생각 중이야.', '분노']
['요즘 청년 실업률이 너무 심각한 거 같아. 취업해야 하는데 기업들이 채용을 많이 하지를 않아.[CLS]기업들이 채용을 늘려야 청년들이 직장을 다니고 경제도 활성화가 될 텐데 채용의 기회조차 줄어드니 화가 나는 거지. 복지도 좋지만 기업 채용을 늘려주는 정책들이 생겼으면 좋겠어.', '분노']
['직장에서 부당한 일을 겪어서 너무 화가 나.[CLS]이미 주변 사람에게 말해봤는데 반응은 다 나보고 그냥 참고 넘어가라는 말밖에 없어. 내 이야기에 공감해 주는 사람이 없어서 더 화가 나는 것 같아

In [11]:
for i in range(len(concatenated_lists)):
    if concatenated_lists[i][1] == '분노':
        concatenated_lists[i][1] = '2'
    elif concatenated_lists[i][1] == '슬픔':
        concatenated_lists[i][1] = '3'
    elif concatenated_lists[i][1] == '기쁨':
        concatenated_lists[i][1] = '5'
    elif concatenated_lists[i][1] == '불안':
        concatenated_lists[i][1] = '7'
        
    

In [12]:
np.random.shuffle(concatenated_lists)

In [13]:
concatenated_lists

[['연금만 바라보고 일했는데 받아보니 생활비로도 쓰기 부족해서 화가 나네.[CLS]이제 어떻게 먹고 살면 좋을지 모르겠어.[CLS]경력을 살려서 재취업을 하는 걸 알아보아야겠어.',
  '2'],
 ['나 진짜 이번 시험 생각하면 짜증나서 돌아버릴 거 같아.[CLS]중간고사 못 봐서 기말은 잘 봐야 하는데 공부도 잘 안되고 심란하기만 해.[CLS]응. 집에서 잘 안돼서 어디라도 다녀야겠어.',
  '2'],
 ['엄마와 아빠가 또 싸우셨는데 나는 아무것도 할 수가 없어서 속상해.[CLS]나는 왜 이런 가정에 태어났을까? 정말 나 자신이 불쌍할 지경이야.[CLS]지금은 무엇을 해야 할지 아무 생각이 안 나.',
  '3'],
 ['남편이 애한테 폭력을 행사했대. 너무 혼란스러워.[CLS]애가 이야기하더라고. 어떡할지 모르겠어.[CLS]아무래도 조만간 금융학과 나온 친구에게 돈 관리 상담을 받는 게 좋겠어.',
  '7'],
 ['오늘 친구랑 싸웠어. 그 친구에게 복수할 거야.[CLS]오늘 친구가 늦게 와서 다음부터는 늦지 말라고 이야기해줬는데 다음 약속에는 나도 늦어버릴 거야.[CLS]그런가? 홧김에 복수하고 싶었나 봐. 그렇게 말해줘서 고마워.',
  '2'],
 ['나이와 신체장애로 인해 퇴직당할 것 같아. 돈도 별로 없는데 이 상황이 불만스러워.[CLS]신체장애와 나이가 많아 무언가를 하기도 힘들어.[CLS]다른 사람에게 속상한 일을 털어놓아야겠어.',
  '7'],
 ['돈이 없어서 자꾸 남편이랑 싸우게 돼.[CLS]너무 짜증 나. 일을 안 나가는 내 잘못이야.[CLS]이런 무가치한 나를 위로해줄 수 있는 사람이 곁에 있으면 좋겠어.',
  '2'],
 ['내 친구 중 한 명이 밥 먹을 때 매번 나에게 얻어먹는데 정말 짜증나.[CLS]한 두 번도 아니고 매번 그러는데 나도 넉넉한 형편이 아니라서 짜증나고 자꾸만 피하게 돼.[CLS]모르겠어. 만나는 것 자체가 불편해서 지금처럼 피해 다닐까 생각 중이야.',
  '2'],
 ['난 몸이 불편해서 할 수 있

In [14]:
# 7개의 감정 class → 숫자
data.loc[(data['상황'] == "fear"), '상황'] = 0  # fear → 0 
data.loc[(data['상황'] == "surprise"), '상황'] = 1  # surprise → 1
data.loc[(data['상황'] == "angry"), '상황'] = 2  # angry → 2 분노
data.loc[(data['상황'] == "sadness"), '상황'] = 3  # sadness → 3 슬픔
data.loc[(data['상황'] == "neutral"), '상황'] = 4  # neutral → 4 
data.loc[(data['상황'] == "happiness"), '상황'] = 5  # happiness → 5 기쁨
data.loc[(data['상황'] == "disgust"), '상황'] = 6  # disgust → 6 

In [15]:
data.value_counts('상황')

상황
5    4548
2    3263
4    3253
3    2848
6    2321
1    1755
0    1386
Name: count, dtype: int64

In [16]:
# [발화문, 상황] data_list 생성
data_list = []
for ques, label in zip (data['발화문'], data['상황']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

In [17]:
data_list = data_list + concatenated_lists
np.random.shuffle(data_list)

data_list

[['나이가 드니까 회사 일을 처리하기가 힘들어.[CLS]젊은 애들에 비해 머리도 안 돌아가고 점점 쓸모없어지는 것 같아.[CLS]내가 할 수 있는 일들에 대해 곰곰이 생각해봐야겠어. 나만 할 수 있는 일.',
  '2'],
 ['여자친구가 해외여행 가고 쇼핑하는 데 돈을 다 써서 결혼 자금이 없대. 정말 실망했어.[CLS]요즘 집값이 얼마인데 너무한 거 아냐? 나는 안 쓰고 안 먹고 안 입고 악착같이 돈을 모았어.',
  '3'],
 ['화장실은 청소를 해도 곰팡이가 없어지지를 않아.', '2'],
 ['개인적으로 연금보험을 들어갔는데 내가 모르고 해약을 해버렸어.[CLS]다달이 돈이 들어왔던 거를 갑자기 왜 해약을 해버린 건지 내가 바보 같아.[CLS]당연히 안 될 거 같아서 물어보지 않았는데 한번 물어 봐야겠어.',
  '7'],
 ['그러게 말이야. 둘 다 신경전이 있었나 봐.', '1'],
 ['아니, 요즘은 다 왜 이러는 거지? 이해가 안 되네.', '2'],
 ['요즘 직장에서 일이 많아서 자주 야근을 해.[CLS]영업이 잘되다 보니 처리해야 할 일들이 많아서 그래.[CLS]돈 많이 버느라 바쁜 거니까 이런 경우라면 매일매일 야근하라면 할 수 있겠어.',
  '5'],
 ['엔터테인먼트 주식이야.', '3'],
 ['어쩔 수 없지. 비가 오니까 나갈 수도 없고. 알았어. 고마워.', '2'],
 ['일주일이나 늦었지만 안전하게 배달 되었어. 꼼꼼하게 살펴볼게.', '5'],
 ['새엄마가 왜 이렇게 눈치가 없냐고 했어. 나 자신이 너무 증오스러워.[CLS]새엄마하고 잘 지내고 싶었는데 내가 잘못했나 봐.[CLS]새엄마에게 내가 어떤 점이 눈치 없었냐고 물어봐야겠어.',
  '2'],
 ['음식점이 가장 많기로 소문난 강남역에서 만나기로 했어.', '4'],
 ['흠. 스트레스 풀만한 데 없나?', '2'],
 ['내가 고졸이라고 연봉이 다른 신입사원보다 적어서 슬퍼.[CLS]실력도 보지 않고 왜 학력으로 연봉을 책정하는 걸까?[CLS]이번에 시작하

In [18]:
m = 0
k = []
for i in data_list:
    if len(i[0]) > m:
        m = len(i[0])
        k = i

k

['남들처럼 배우질 못해서 몸만 믿고 살아왔는데 다리를 다쳤으니 어떡한담. 종일 쉬엄쉬엄 걸어 다녔는데 그걸 못하면 풀칠하기도 힘들 텐데.[CLS]건강기능식품 같은 걸 파는 외판원을 하고 있는데 나이에 구애도 받지 않고 부지런만 하면 굶어 죽지는 않거든. 얼마 전에는 단골들이랑 산에 올랐다가 미끄러져서 병원을 찾았더니 의사가 앞으로는 한 시간 이상 걷지도 말라잖아.[CLS]하루 벌어 하루 풀칠하는 형편이니 차가 있을 리도 없지만 차를 누가 주더라도 운전할 줄이나 알겠어? 다리뿐 아니라 귀도 나쁘고 눈도 나쁘고 나쁜 데가 한두 군데가 아니니 죽는 게 나을지도 모르지',
 '2']

In [19]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)
print(len(dataset_train), len(dataset_test))

42484 10621


In [20]:
# parameter 값 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [21]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False) # 사전학습된 KoBERT 가져오기
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

#저는 csv 파일을 사용하여 이부분은 dataset_train.values 로 해주었습니다. 
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)


/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [22]:
# torch 형식의 dataset을 만들어주면서, 입력 데이터셋의 전처리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [23]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 8,   # 클래스 수 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [24]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [25]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [27]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/tmp/ipykernel_133910/4005747619.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/664 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.216826915740967 train acc 0.03125
epoch 1 batch id 201 loss 1.532107949256897 train acc 0.21120957711442787
epoch 1 batch id 401 loss 0.922720193862915 train acc 0.39506701995012466
epoch 1 batch id 601 loss 0.8027206659317017 train acc 0.5112312811980033
epoch 1 train acc 0.5361029454355886


/tmp/ipykernel_133910/4005747619.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/166 [00:00<?, ?it/s]

...

epoch 19 test acc 0.8301356038909736


  0%|          | 0/664 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.0004515045729931444 train acc 1.0
epoch 20 batch id 201 loss 0.001110866549424827 train acc 0.996268656716418
epoch 20 batch id 401 loss 0.0005057590315118432 train acc 0.996376246882793
epoch 20 batch id 601 loss 0.0004482982913032174 train acc 0.9967762063227953
epoch 20 train acc 0.9968702936746988


  0%|          | 0/166 [00:00<?, ?it/s]

epoch 20 test acc 0.8304179834090459


In [28]:
# predict : 학습 모델을 활용하여 다중 분류된 클래스를 출력해주는 함수
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)
tok = tokenizer.tokenize 

def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")
            elif np.argmax(logits) == 7:
                test_eval.append("불안이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [29]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

In [30]:
text_a = '한국어 모델을 공유합니다.'
tokens_1 = tokenizer.tokenize(text_a)
tokens_2 = tokenizer(text_a)
print(tokens_1, type(tokens_1))
print(tokens_2, type(tokens_2))

['▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.'] <class 'list'>
{'input_ids': [2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]} <class 'transformers.tokenization_utils_base.BatchEncoding'>


In [31]:
# 모델 저장 경로 설정
model_save_path = './saved_model'

# 모델의 가중치 저장
torch.save(model.state_dict(), model_save_path + '/pytorch_model.bin')

# 토크나이저 저장
tokenizer.save_pretrained(model_save_path)

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/spiece.model',
 './saved_model/added_tokens.json')